# Introduction

In this notebook you will find practical examples for every part of the theoretical part. Including how to build and train deep neural networks.

You will also find questions that are designed to help you notice important concepts and implementation details.

In [3]:
import easydict

##Calibrate
# pylint: disable=E1129,E0611,E1101
import argparse
import datetime
import random
from tqdm import tqdm
# Hide warnings before importing tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#import tensorflow as tf
import pandas as pd
#from . import reformat
#from .load_data import split_features_labels, list_categorical_columns
#from .adjustments import Scaler, SigmoidScaler, split_discrete_continuous
#from .network import Confounded

#Adjustments
import numpy as np
import pandas as pd

#Network
# pylint: disable=E1129
!pip install tf-slim
import functools
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#import tensorflow as tf
#from tensorflow.contrib.layers import fully_connected, batch_norm # pylint: disable=E0611
from tf_slim.layers.layers import fully_connected, batch_norm
from math import ceil

In [4]:
#Load google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
dirc = '/content/gdrive/MyDrive/AI in Microbiome/Filtered normalized data/noBE_removal_clr_and_other_normalizations/MAD/'
dirc = '/content/gdrive/MyDrive/PhD/Coursework/AI in Genomics/Projects/Phyla/phyla_dataset_d3/'
file1 = 'phyla_all_3949x1177_pmi_0_clr_75p_MAD.csv'
file2 = 'phyla_all_565x1177_pmi_0_clr_10p_MAD.csv'
file3 = 'phyla_all_753x1177_pmi_0_clr_15p_MAD.csv'
file4 = 'phyla_biopsy_1109x1177_pmi_0_clr_75p_MAD.csv'
file5 = 'phyla_biopsy_164x1177_pmi_0_clr_10p_MAD.csv'
file6 = 'phyla_biopsy_213x1177_pmi_0_clr_15p_MAD.csv'
file7 = 'phyla_stool_2840x1177_pmi_0_clr_75p_MAD.csv'
file8 = 'phyla_stool_401x1177_pmi_0_clr_10p_MAD.csv'
file9 = 'phyla_stool_540x1177_pmi_0_clr_15p_MAD.csv'
file10 = 'phyla_dataset_d3.csv'

file_used = file10
#os.listdir(dirc)

In [6]:
microbe_df = pd.read_csv(F'%s' %(dirc+file_used))

In [7]:
#Determine size of dataframe
print(microbe_df.shape)
microbe_df.iloc[1:2,1177:1184]

(5267, 1184)


,Unassigned;__;__;__;__;__,col_site,diagnosis,sample_title,stool_biopsy,studyID,uc_cd
1,0.0,OSCCAR,1,1939.100003,stool,GEVERSM,CD


In [ ]:
#Subset df
#microbe_df = microbe_df.loc[microbe_df['uc_cd' ] == 'Control']

In [8]:
#Rename columns and col_site
phenotype_df = microbe_df
phenotype_df.loc[(phenotype_df.col_site == 'AG'), 'col_site'] = 1
phenotype_df.loc[(phenotype_df.col_site == 'North Carolina'), 'col_site'] = 2
phenotype_df.loc[(phenotype_df.col_site == 'BOULDER'), 'col_site'] = 3
phenotype_df.loc[(phenotype_df.col_site == 'RISK'), 'col_site'] = 4
phenotype_df.loc[(phenotype_df.col_site == 'paneth_cells'), 'col_site'] = 5
phenotype_df.loc[(phenotype_df.col_site == 'CVDF'), 'col_site'] = 6
phenotype_df.loc[(phenotype_df.col_site == 'Cincinnati'), 'col_site'] = 7
phenotype_df.loc[(phenotype_df.col_site == 'MGH'), 'col_site'] = 8
phenotype_df.loc[(phenotype_df.col_site == 'PRISM'), 'col_site'] = 9
phenotype_df.loc[(phenotype_df.col_site == 'MGH Pediatrics'), 'col_site'] = 10
phenotype_df.loc[(phenotype_df.col_site == 'Cedars-Sinai'), 'col_site'] = 11
phenotype_df.loc[(phenotype_df.col_site == 'PRJNA436359'), 'col_site'] = 12
phenotype_df.loc[(phenotype_df.col_site == 'OSCCAR'), 'col_site'] = 13

In [9]:
#Rename col_site
phenotype_df.rename(columns={'sample_title':'sample'}, inplace=True)

In [10]:
#Change sample ids to a vector of integers
phenotype_df['sample'] = list(range(0,phenotype_df.shape[0],1))

In [11]:
phenotype_df.col_site.value_counts()

1     1279
4     1210
12    1178
2      516
3      467
9      260
13      91
5       82
7       55
8       45
6       39
11      25
10      20
Name: col_site, dtype: int64

In [12]:
phenotype_df.to_csv('/content/gdrive/MyDrive/PhD/Coursework/AI in Genomics/VAE/' + file_used.replace('.csv','_edited.csv'), index = False)

In [13]:
#Read data file
input = '/content/gdrive/MyDrive/PhD/Coursework/AI in Genomics/VAE/' + file_used.replace('.csv','_edited.csv')
output = '/content/gdrive/MyDrive/PhD/Coursework/AI in Genomics/VAE/' + file_used.replace('.csv','_edited_vae.csv')
logfile = '/content/gdrive/MyDrive/PhD/Coursework/AI in Genomics/VAE/vae_log.csv'

##Main: Calibrate
https://github.com/jdayton3/Confounded/blob/master/confounded/reformat.py

In [19]:
args = easydict.EasyDict({
    "file" : input,           #Path to input file
    "source_file" : input,    #Path to input file
    "output_file" : output,   #Path to output file. (default: `source-file`_confounded.csv)
    "minibatch_size" : 100, #The size of the mini-batch for training. Must be positive integer."
    "layers" : 10,          #How many layers deep the discriminator should be. Must be positive integer.
    "ae_layers" : 2,        #How many layers in each of the encoding and decoding portions of the autoencoder.
    "batch_col" : "col_site",  #Which column contains the batch to adjust for
    "code_size" : 20,       #How many nodes in the code layer of the autoencoder."
    "early_stopping" : None, #type=positive_int, default=None, help="How many iterations without improvement before stopping early.
    "scaling" : "linear",   #choices=["linear", "sigmoid"], default="linear", help="Type of scaling to perform on the input data.
    "loss_weight" : 2.0,    #Weight applied to the discriminator loss when training the autoencoder.
    "log_file" : logfile, #Path to file to log results
    "iterations" : 10000,   #Number of iterations of minibatches to run
    "save_model" : None,    #Path to save the model weights. Weights are not saved if path is not specified
    "load_model" : None,    #help="Path to model weights checkpoint to load.", " Weights are initialized randomly if path is not specified.")
    "learning_rate" : 0.0001, #, type=float, 
})

In [20]:

def parse_arguments():
    activation = tf.nn.relu

    def positive_int(value):
        ivalue = int(value)
        if ivalue < 0:
            raise argparse.ArgumentTypeError("%s is an invalid positive int value" % value)
        return ivalue
    '''
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    parser.add_argument(
        'file', metavar='source-file', type=str,
        help='Path to input file.')
    parser.add_argument(
        '-o', "--output-file", type=str,
        help="Path to output file. (default: `source-file`_confounded.csv)")
    parser.add_argument(
        "-m", "--minibatch-size", type=positive_int, default=100,
        help="The size of the mini-batch for training. Must be positive integer.")
    parser.add_argument(
        "-l", "--layers", type=positive_int, default=10,
        help="How many layers deep the discriminator should be. Must be positive integer.")
    parser.add_argument(
        "-a", "--ae-layers", type=positive_int, default=2,
        help="How many layers in each of the encoding and decoding portions of the autoencoder.")
    parser.add_argument(
        "-b", "--batch-col", type=str, default="Batch",
        help="Which column contains the batch to adjust for.")
    parser.add_argument(
        "-c", "--code-size", type=positive_int, default=20,
        help="How many nodes in the code layer of the autoencoder.")
    parser.add_argument(
        "-e", "--early-stopping", type=positive_int, default=None,
        help="How many iterations without improvement before stopping early.")
    parser.add_argument(
        "-s", "--scaling", choices=["linear", "sigmoid"], default="linear",
        help="Type of scaling to perform on the input data.")
    parser.add_argument(
        "-w", "--loss-weight", type=float, default=1.0,
        help="Weight applied to the discriminator loss when training the autoencoder.")
    parser.add_argument(
        "-f", "--log-file", type=str, default="./data/metrics/log.csv",
        help="Path to file to log results.")
    parser.add_argument(
        "-i", "--iterations", type=positive_int, default=10000,
        help="Number of iterations of minibatches to run.")
    parser.add_argument(
        "-d", "--save-model", type=str, default=None,
        help="Path to save the model weights. Weights are not saved if path is not specified.")
    parser.add_argument(
        "-r", "--load-model", type=str, default=None,
        help="Path to model weights checkpoint to load."
            " Weights are initialized randomly if path is not specified.")
    parser.add_argument(
        "-g", "--learning-rate", type=float, default=0.0001,
        help="Global learning rate for all portions of the network.")

    args = parser.parse_args()
    '''
    if not args.output_file:
        args.output_file = args.file.rstrip(".csv") + "_confounded.csv"
    args.activation = activation

    return args

def now():
    """Return the current time in iso format"""
    return datetime.datetime.now().isoformat()

def should_train_dual(i, tot_iterations):
    """For a given iteration, should we train ae & disc?
    Cools down over time.
    """
    probability = 1.0 - i / tot_iterations
    probability = 0.1
    return random.uniform(0.0, 1.0) < probability

class SummaryLogger(object):
    def __init__(self,
                 log_file,
                 input_path,
                 output_path,
                 code_size,
                 d_layers,
                 ae_layers,
                 minibatch_size,
                 activation,
                 batch_col,
                 scaling,
                 loss_weight):
        self.start_time = now()
        self.log_file = log_file
        self.input_path = input_path
        self.output_path = output_path
        self.code_size = code_size
        self.d_layers = d_layers
        self.ae_layers = ae_layers
        self.minibatch_size = minibatch_size
        self.activation = activation
        self.batch_col = batch_col
        self.scaling = scaling
        self.loss_weight = loss_weight
        self.values = {
            "start_time": [],
            "input_path": [],
            "output_path": [],
            "batch_column": [],
            "minibatch_size": [],
            "code_size": [],
            "discriminator_layers": [],
            "autoencoder_layers": [],
            "activation": [],
            "scaling_method": [],
            "loss_weight": [],
            "time": [],
            "iteration": [],
            "ae_loss": [],
            "disc_loss": [],
            "dual_loss": [],
        }

    def log(self, iteration, ae_loss, disc_loss, dual_loss):
        self.values["start_time"].append(self.start_time)
        self.values["input_path"].append(self.input_path)
        self.values["output_path"].append(self.output_path)
        self.values["batch_column"].append(self.batch_col)
        self.values["minibatch_size"].append(self.minibatch_size)
        self.values["code_size"].append(self.code_size)
        self.values["discriminator_layers"].append(self.d_layers)
        self.values["autoencoder_layers"].append(self.ae_layers)
        self.values["activation"].append(".".join([
            self.activation.__module__,
            self.activation.__name__
        ]))
        self.values["scaling_method"].append(self.scaling)
        self.values["loss_weight"].append(self.loss_weight)
        self.values["time"].append(now())
        self.values["iteration"].append(iteration)
        self.values["ae_loss"].append(ae_loss)
        self.values["disc_loss"].append(disc_loss)
        self.values["dual_loss"].append(dual_loss)

    def save(self):
        new_df = pd.DataFrame(self.values)
        old_df = None
        try:
            old_df = pd.read_csv(self.log_file)
        except:
            pass
        if old_df is not None:
            new_df = pd.concat([old_df, new_df], ignore_index=True, sort=False)
        new_df.to_csv(self.log_file, index=False)

def autoencoder(input_path,
                output_path,
                save_weights_path,
                load_weights_path,
                minibatch_size=100,
                code_size=200,
                iterations=10000,
                d_layers=2,
                ae_layers=2,
                activation=tf.nn.relu,
                batch_col="Batch",
                early_stopping=None,
                scaling="linear",
                disc_weighting=1.0,
                log_file="log.csv",
                learning_rate=0.0001):

    # Get sizes & meta cols
    print("Loading data")
    data = pd.read_csv(input_path)
    original_cols = data.columns.tolist()

    scaling_options = {
        "linear": Scaler,
        "sigmoid": SigmoidScaler
    }

    scaler = scaling_options[scaling]()
    data = scaler.squash(data)

    meta_cols = list_categorical_columns(data)
    print("Inferred meta columns: {}. These will not be adjusted.".format(meta_cols))

    input_size = len(data.columns) - len(meta_cols)
    num_targets = len(data[batch_col].unique())

    c = Confounded(
        input_size,
        code_size,
        num_targets,
        discriminator_layers=d_layers,
        autoencoder_layers=ae_layers,
        activation=activation,
        disc_weghting=disc_weighting
    )

    all_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    vars_to_replace = [var for var in all_vars if "do_not_save" in var.name]
    vars_to_keep = [var for var in all_vars if var not in vars_to_replace]
    saver = tf.train.Saver(vars_to_keep)

    with tf.Session() as sess:
        if load_weights_path:
            saver.restore(sess, load_weights_path)
            print("Model loaded from path: {}".format(load_weights_path))
            tf.variables_initializer(vars_to_replace).run()
        else:
            tf.global_variables_initializer().run()
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter("log/{}".format(now()), sess.graph)

        logger = SummaryLogger(
            log_file,
            input_path,
            output_path,
            code_size,
            d_layers,
            ae_layers,
            minibatch_size,
            activation,
            batch_col,
            scaling,
            disc_weighting
        )
        # Train
        n_since_improvement = 0
        best_loss = float("inf")

        print("Training Confounded (learning rate={})".format(learning_rate))
        for i in tqdm(range(iterations)):
            features, labels = split_features_labels(
                data,
                batch_col,
                meta_cols=meta_cols,
                sample=minibatch_size
            )
            summary, disc_loss, ae_loss, dual_loss, _, _, _ = sess.run([
                merged,
                c.d_loss,
                c.ae_loss,
                c.loss,
                c.outputs,
                c.optimizer,
                c.d_optimizer,
            ], feed_dict={
                c.inputs: features,
                c.targets: labels,
            })

            writer.add_summary(summary, i)
            logger.log(i, ae_loss, disc_loss, dual_loss)
            if dual_loss < best_loss:
                n_since_improvement = 0
                best_loss = dual_loss
            else:
                n_since_improvement += 1
            if early_stopping is not None and n_since_improvement >= early_stopping:
                print("No loss improvement for {} iterations. Stopping at iteration {}.".format(
                    n_since_improvement,
                    i
                ))
                break

        logger.save()
        if save_weights_path:
            saver.save(sess, save_weights_path)
            print("Model saved in path: {}".format(save_weights_path))

        print("Adjusting the input data")
        features, labels = split_features_labels(data, batch_col, meta_cols=meta_cols)
        adj, = sess.run([c.outputs], feed_dict={
            c.inputs: features,
            c.targets: labels,
        })
        print("Saving data to {}".format(output_path))
        df_adj = pd.DataFrame(adj, columns=split_discrete_continuous(data)[-1].columns)
        df_adj = scaler.unsquash(df_adj)

        to_csv(
            df_adj,
            output_path,
            tidy=True,
            meta_cols={
                col: data[col] for col in meta_cols
            },
            column_order=original_cols
        )

if __name__ == "__main__":

    args = parse_arguments()

    autoencoder(
        args.file,
        args.output_file,
        args.save_model,
        args.load_model,
        args.minibatch_size,
        args.code_size,
        args.iterations,
        d_layers=args.layers,
        ae_layers=args.ae_layers,
        activation=args.activation,
        batch_col=args.batch_col,
        early_stopping=args.early_stopping,
        scaling=args.scaling,
        disc_weighting=args.loss_weight,
        log_file=args.log_file,
        learning_rate=args.learning_rate
    )

Loading data
Inferred meta columns: ['run_accession', 'col_site', 'diagnosis', 'sample', 'stool_biopsy', 'studyID', 'uc_cd']. These will not be adjusted.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
  0%|          | 0/10000 [00:00<?, ?it/s]

Training Confounded (learning rate=0.0001)


100%|██████████| 10000/10000 [19:30<00:00,  8.54it/s]


Adjusting the input data
Saving data to /content/gdrive/MyDrive/PhD/Coursework/AI in Genomics/VAE/phyla_dataset_d3_edited_vae.csv


##Adjustments

In [18]:
"""Adjustments to dataframes"""

def split_discrete_continuous(df):
    discrete_types = ['object', 'int']
    discrete = df.select_dtypes(include=discrete_types)
    continuous = df.select_dtypes(exclude=discrete_types)
    return discrete, continuous

class Scaler(object):
    """Scale or unscale a dataframe from [min, max] <-> [0, 1]
    """
    def __init__(self):
        self.col_min = None
        self.col_max = None
        self.squashed = False

    def squash(self, df):
        """Adjust the dataframe to the [0, 1] range.
        Arguments:
            df {pandas.DataFrame} -- The quantitative dataframe to be
                squashed.
        Returns:
            pandas.DataFrame -- The squashed dataframe.
        """
        discrete, continuous = split_discrete_continuous(df)
        scaled = self._squash_continuous(continuous)
        self.squashed = True
        return pd.concat([discrete, scaled], axis="columns")

    def _squash_continuous(self, continuous):
        self.col_min = continuous.min()
        self.col_max = continuous.max()
        already_in_range = (
            (self.col_min >= 0.0) &
            (self.col_min <= 1.0) &
            (self.col_max >= 0.0) &
            (self.col_max <= 1.0)
        )
        self.col_min = np.where(already_in_range, 0.0, self.col_min)
        self.col_max = np.where(already_in_range, 1.0, self.col_max)

        scaled = (continuous - self.col_min) / (self.col_max - self.col_min)
        return scaled

    def unsquash(self, df):
        """Adjust the dataframe back to the original range.
        Arguments:
            df {pandas.DataFrame} -- The quantitative dataframe to be
                expanded.
        Returns:
            pandas.DataFrame -- The dataframe with each column expanded
                to its original range.
        """
        if not self.squashed:
            raise Exception(
                "Error: Scaler.squash() must be run "
                "before Scaler.unsquash() can be used."
            )
        discrete, continuous = split_discrete_continuous(df)
        scaled = self._unsquash_continuous(continuous)
        if discrete.size == 0:
            return scaled
        return pd.concat([discrete, scaled], axis="columns")

    def _unsquash_continuous(self, continuous):
        scaled = continuous * (self.col_max - self.col_min) + self.col_min
        return scaled

class SigmoidScaler(Scaler):
    """Scaler class that scales continuous values into (0.0, 1.0) using
    the sigmoid function.
    """
    def _squash_continuous(self, continuous):
        return self.__sigmoid(continuous)

    def _unsquash_continuous(self, continuous):
        return self.__logit(continuous)

    def __sigmoid(self, continuous):
        return 1.0 / (1.0 + np.exp(-continuous))

    def __logit(self, continuous):
        """inverse of sigmoid"""
        return np.log(continuous) - np.log(1 - continuous)


##Load data

In [17]:

def split_features_labels(df, batch_col, meta_cols=None, sample=None):
    """Split a dataframe into features and labels numpy arrays.
    Arguments:
        df {pandas.DataFrame} -- A tidy dataframe with meta columns, a
            Batch column, and quantitative data.
    Keyword Arguments:
        meta_cols {list of strings} -- Columns that should not be
            used as features to be batch-adjusted (default:
            {["Sample", `batch_col`]})
        sample {int} -- The number of rows to sample. If None, return
            all rows. (default: {None})
    Returns:
        [(numpy.array, numpy.array)] -- Tuple of features and labels,
            where features are the quantitative data from the given
            dataframe and labels are the one-hot encoded batches for
            each instance.
    """
    if meta_cols is None:
        meta_cols = ["Sample", batch_col]
    features = np.array(df.drop(meta_cols, axis=1))
    labels = pd.get_dummies(df[batch_col])
    labels = np.array(labels, dtype=float)
    if sample is not None:
        rows = pd.DataFrame(df.index).sample(sample, replace=True)[0].values.tolist()
        features = features[rows]
        labels = labels[rows]
    return features, labels

def list_categorical_columns(df):
    """Get the names of all categorical columns in the dataframe.
    Arguments:
        df {pandas.DataFrame} -- The dataframe.
    Returns:
        list -- Names of the categorical columns in the dataframe.
    """
    return list(df.select_dtypes(exclude=['float']).columns)

##Network

In [16]:
"""Definitions for the neural networks in Confounded.
"""

def is_square(n):
    sqrt = n**0.5
    return int(sqrt) == sqrt

def var_scope(scope):
    """Decorator to wrap a function in a tensorflow variable scope
    Arguments:
        scope {str} -- Name of the variable scope
    Returns:
        function -- The decorated function wrapped in the variable scope
    """
    def decorator_var_scope(func):
        @functools.wraps(func)
        def wrapper_var_scope(*args, **kwargs):
            with tf.variable_scope(scope):
                return func(*args, **kwargs)
        return wrapper_var_scope
    return decorator_var_scope

def show_before_and_after_images(func):
    @functools.wraps(func)
    def wrapper_show_images(*args, **kwargs):
        inputs = args[0]
        show_image(inputs, name="inputs")
        outputs = func(*args, **kwargs)
        if isinstance(outputs, tuple):
            # The autoencoder functions might return (outputs, loss)
            show_image(outputs[0], name="outputs")
        else:
            show_image(outputs, name="outputs")
        return outputs
    return wrapper_show_images

@var_scope("vae")
@show_before_and_after_images
def variational_autoencoder(inputs, code_size=20):
    """Creates a variational autoencoder based on "Hands-On Machine
    Learning with Scikit-Learn and TensorFlow by Aurélien Géron
    (O’Reilly). Copyright 2017 Aurélien Géron, 978-1-491-96229-9."
    Arguments:
        input_size {int} -- Size of the input to the autoencoder
    Returns:
        input {Tensor} -- The input tensor
        output {Tensor} -- The output tensor
        loss {Tensor} -- The loss operation
    """
    layer_sizes = [500, 500]
    activations = [tf.nn.elu for _ in layer_sizes]
    input_size = get_layer_size(inputs)

    encoding = make_layers(inputs, layer_sizes, activations)
    code_mean, code_gamma, code = vae_code_layer(encoding, code_size)
    decoding = make_layers(code, layer_sizes, activations)
    logits = fully_connected(decoding, input_size, activation_fn=None)
    outputs = tf.sigmoid(logits)

    reconstruction_loss = tf.losses.mean_squared_error(inputs, outputs)
    xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=inputs, logits=logits)
    reconstruction_loss = tf.reduce_sum(xentropy)
    latent_loss = kl_divergence(code_gamma, code_mean) #* 0.01 / code_size
    loss = reconstruction_loss + latent_loss

    loss = loss / input_size

    return outputs, loss

def get_layer_size(layer):
    dimensions = layer.shape[1:]
    size = 1
    for dimension in dimensions:
        size *= int(dimension) # must be converted from Dimension to int
    return size

def make_layers(inputs, layer_sizes, activations=None, keep_prob=1.0, do_batch_norm=False):
    if not activations:
        activations = [tf.nn.relu for _ in layer_sizes]
    current_layer = inputs
    for layer_size, activation in zip(layer_sizes, activations):
        current_layer = fully_connected(current_layer, layer_size, activation_fn=activation)
        current_layer = tf.nn.dropout(current_layer, keep_prob)
        if do_batch_norm:
            current_layer = batch_norm(current_layer)
    return current_layer

def vae_code_layer(inputs, code_size):
    code_mean = fully_connected(inputs, code_size, activation_fn=None)
    code_gamma = fully_connected(inputs, code_size, activation_fn=None)
    noise = tf.random_normal(tf.shape(code_gamma), dtype=tf.float32)
    code = code_mean + tf.exp(0.5 * code_gamma) * noise
    return code_mean, code_gamma, code

def kl_divergence(gamma, mean):
    return 0.5 * tf.reduce_sum(tf.exp(gamma) + tf.square(mean) - 1 - gamma)

def show_image(x, name="image"):
    input_size = get_layer_size(x)
    if is_square(input_size):
        width_height = int(input_size**0.5)
        img = tf.reshape(x, [-1, width_height, width_height, 1])
        tf.summary.image(name, img, max_outputs=1)

class Confounded(object):
    def __init__(self,
                 input_size,
                 code_size,
                 num_targets,
                 discriminator_layers=2,
                 autoencoder_layers=2,
                 activation=tf.nn.relu,
                 disc_weghting=1.0,
                 learning_rate=0.0001):
        self.sess = tf.Session() #tf.compat.v1.Session() #tf.Session()

        self.input_size = input_size
        self.code_size = code_size
        self.num_targets = num_targets
        self.discriminator_layers = discriminator_layers
        self.autoencoder_layers = autoencoder_layers
        self.activation = activation
        self.disc_weighting = disc_weghting
        self.learning_rate = learning_rate

        self.inputs = None
        self.code = None
        self.outputs = None
        self.targets = None
        self.logits = None
        self.classification = None

        self.d_loss = None
        self.ae_loss = None
        self.loss = None
        self.optimizer = None
        self.d_optimizer = None

        self._setup_networks()

    def _setup_networks(self):
        self._setup_autoencoder()
        self._setup_discriminator()
        self._setup_loss_functions()

    @var_scope("autoencoder")
    def _setup_autoencoder(self):
        self.inputs = tf.placeholder(tf.float32, [None, self.input_size])
        self.outputs, self.ae_loss = variational_autoencoder(self.inputs, code_size=self.code_size)

    @var_scope("discriminator")
    def _setup_discriminator(self):
        self.targets = tf.placeholder(tf.float32, [None, self.num_targets])
        inputs = batch_norm(self.outputs)
        layer_size = 512
        layer_sizes = [int(ceil(layer_size / 2**n)) for n in range(self.discriminator_layers)]
        layer_sizes = [1024, 512, 512, 128]
        penultimate_layer = make_layers(self.outputs, layer_sizes, keep_prob=0.5, do_batch_norm=True)
        with tf.variable_scope("do_not_save"):
            self.logits = fully_connected(penultimate_layer, self.num_targets, activation_fn=None)
            self.classification = tf.nn.sigmoid(self.logits)

    @var_scope("discriminator")
    @var_scope("optimizer")
    def _setup_disc_loss(self):
        xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.targets, logits=self.logits)
        self.d_loss = tf.reduce_sum(xentropy) / self.num_targets
        tf.summary.scalar("d_loss", self.d_loss)
        discriminator_vars = tf.get_collection(
            tf.GraphKeys.TRAINABLE_VARIABLES,
            "discriminator"
        )
        self.d_optimizer = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate,
            name="discriminator"
        ).minimize(self.d_loss, var_list=discriminator_vars)

    @var_scope("autoencoder")
    @var_scope("optimizer")
    def _setup_ae_loss(self):
        tf.summary.scalar("ae_loss", self.ae_loss)
        autoencoder_vars = tf.get_collection(
            tf.GraphKeys.TRAINABLE_VARIABLES,
            "autoencoder"
        )
        self.ae_optimizer = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate,
            name="ae"
        ).minimize(self.ae_loss, var_list=autoencoder_vars)

    @var_scope("autoencoder")
    @var_scope("optimizer")
    def _setup_dual_loss(self):
        autoencoder_vars = tf.get_collection(
            tf.GraphKeys.TRAINABLE_VARIABLES,
            "autoencoder"
        )
        self.loss = self.ae_loss - self.disc_weighting * self.d_loss
        tf.summary.scalar("dual_loss", self.loss)
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate,
            name="dual"
        ).minimize(self.loss, var_list=autoencoder_vars)

    def _setup_loss_functions(self):
        self._setup_disc_loss()
        self._setup_ae_loss()
        self._setup_dual_loss()

##Reformat

In [15]:

def to_csv(df, path, tidy=False, meta_cols=None, column_order=None):
    """Save a dataframe as a CSV, and optionally add meta columns and
        transpose the dataframe.
    Arguments:
        df {pandas.DataFrame} -- The data to be saved.
        path {str} -- The path where the csv should be saved.
    Keyword Arguments:
        tidy {bool} -- True if the rows are instances and the columns
            are features; False otherwise. (default: {False})
        meta_cols {dict} -- Dictionary of {column_name: [values, ...]}
            (default: {None})
    """
    if not tidy:
        df = df.T
    df = _add_meta_cols(df, meta_cols)

    if column_order:
        df = df[column_order]

    df.to_csv(path, index=None)

def _add_meta_cols(df, meta_cols):
    if meta_cols is None:
        meta_cols = {}
    sample = meta_cols.get("Sample")
    df['Sample'] = _check_sample(df, sample)
    for col_name, col in meta_cols.items():
        if col_name in ["Sample"]:
            continue
        df[col_name] = col
    return _reorder_cols(df, list(meta_cols.keys()))

def _check_sample(df, sample):
    if sample is None:
        return df.index
    return sample

def _reorder_cols(df, meta_cols):
    cols = df.columns.tolist()
    for col in meta_cols:
        cols.remove(col)
    cols = meta_cols + cols
    return df[cols]

if __name__ == "__main__":
    import functools
    import operator
    '''
    in_path = "data/mnist_matrix.csv"
    out_path = "data/tidy_mnist2.csv"
    df = pd.read_csv(in_path, header=None)
    to_csv(df, out_path, tidy=False, meta_cols={
        "Batch": None,
        "Sample": None,
        "Digit": functools.reduce(
            operator.add,
            [[x] * 1000 for x in range(10)]
        )
    })
    '''